# RabbitMQ worker - basics and getting it working

Exploration of how to create a worker that talks to RabbitMQ.

We will later chunk this up into relevant components and bases for the main application. 

Prerequisites: The RabbitMQ docker container is up and running with port 5672 open.  Here is the docker-compose.yml file that I used to start the container:

```yml
  rabbit:
    image: rabbitmq:management
    restart: always
    ports:
      - 5672:5672
      - 15672:15672
    environment:
      RABBITMQ_DEFAULT_USER: DEV_USER  # TEST ONLY
      RABBITMQ_DEFAULT_PASS: CHANGE_ME  # TEST ONLY
    logging:
      driver: local
```

In [1]:
import pika 

In [2]:
# credential for RabbitMQ
credentials = pika.PlainCredentials('DEV_USER', 'CHANGE_ME')

The following is a simple publisher that will send 3 messages to the queue `hello`.

In [3]:
import pika
import time

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost', credentials=credentials))
channel = connection.channel()

channel.queue_declare(queue='hello')

for i in range(3):
    message = 'Hello World! %d' % i
    channel.basic_publish(exchange='',
                          routing_key='hello',
                          body=message)
    print(" [x] Sent %r" % message)
    time.sleep(1)

connection.close()


 [x] Sent 'Hello World! 0'
 [x] Sent 'Hello World! 1'
 [x] Sent 'Hello World! 2'


The following is a simple worker that will listen for messages on the queue `hello` and print them out.  The worker will exit after 10 messages.

In [ ]:
import pika
import time

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost', credentials=credentials))
channel = connection.channel()


channel.queue_declare(queue='hello')

def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    ch.basic_ack(delivery_tag = method.delivery_tag)
    
channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='hello', on_message_callback=callback)

print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C
 [x] Received b'Hello World! 0'
 [x] Done
 [x] Received b'Hello World! 1'
 [x] Done
 [x] Received b'Hello World! 2'
 [x] Done


OK this works.  The former sends messages.  The latter picks them up.  The latter blocks the thread and that's OK.

Next we will refactor this into a suitable structure for base and components in Polylith ...